# সিফ্ট কেরাস এর সাথে ব্যক্তিগত ভবিষ্যদ্বাণী  (Private Predictions with Syft Keras)

# পর্ব  1: পাবলিক প্রশিক্ষণ (Public Training)

এই টিউটোরিয়ালে স্বাগতম! নিম্নলিখিত নোটবুকগুলিতে আপনি কীভাবে ব্যক্তিগত পূর্বাভাস সরবরাহ করবেন তা শিখবেন। ব্যক্তিগত ভবিষ্যদ্বাণী অনুসারে, আমরা বোঝাতে চাইছি যে পুরো প্রক্রিয়া জুড়ে ডেটা অবিচ্ছিন্নভাবে এনক্রিপ্ট করা থাকে। কোনও পর্যায়ে ব্যবহারকারী কাঁচা ডেটা ভাগ করে না, কেবল এনক্রিপ্ট করে (এটি গোপন ভাগ করে নেওয়া হয়)। এই ব্যক্তিগত ভবিষ্যদ্বাণী সরবরাহ করার জন্য, সিফ্ট কেরাস নামক একটি লাইব্রেরি ব্যবহার করে [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) অধীনে টিএফ এনক্রিপটেডটি কাটিং-এজ-ক্রিপ্টোগ্রাফিক এবং মেশিন লার্নিংয়ের কৌশলগুলিকে একত্রিত করে, তবে আপনাকে এটি নিয়ে চিন্তা করতে হবে না এবং আপনার মেশিন লার্নিং অ্যাপ্লিকেশনটিতে ফোকাস করতে পারেন।

আপনি কেবলমাত্র তিনটি পদক্ষেপের সাহায্যে ব্যক্তিগত ভবিষ্যদ্বাণী দেওয়া শুরু করতে পারেন:
- **পর্ব  1**: আপনার মডেলটিকে সাধারণ কেরাস দিয়ে প্রশিক্ষণ দিন।
- **পর্ব  2**: আপনার মেশিন লার্নিং মডেল (সার্ভার) সুরক্ষিত এবং পরিবেশন করুন।
- **পর্ব  3**: ব্যক্তিগত ভবিষ্যদ্বানী (ক্লায়েন্ট) পেতে সুরক্ষিত মডেলটি জিজ্ঞাসা করুন।

ঠিক আছে, আসুন এই তিনটি পদক্ষেপটি অতিক্রম করুন যাতে আপনি ব্যবহারকারীর গোপনীয়তা বা মডেল সুরক্ষা ত্যাগ না করে কার্যকর মেশিন লার্নিং পরিষেবা স্থাপন করতে পারেন।

লেখক:
- Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)
- Yann Dupis - Twitter: [@YannDupis](https://twitter.com/YannDupis)
- Morten Dahl - Twitter: [@mortendahlcs](https://github.com/mortendahlcs)

সমর্থনে:
- Dropout Labs - Twitter: [@dropoutlabs](https://twitter.com/dropoutlabs)
- TF Encrypted - Twitter: [@tf_encrypted](https://twitter.com/tf_encrypted)

অনুবাদক:

- Sourav Das - Twitter: [@adventuroussrv](https://twitter.com/adventuroussrv)

## কেরাসের সাথে আপনার মডেলটিকে প্রশিক্ষণ দিন (Train your model with Keras)

আপনার প্রকল্পগুলির জন্য গোপনীয়তা সংরক্ষণের মেশিন শেখার কৌশলগুলি ব্যবহার করার জন্য আপনাকে একটি নতুন মেশিন লার্নিং ফ্রেমওয়ার্ক শিখতে হবে না। আপনার যদি বেসিক থাকে[Keras](https://keras.io/) জ্ঞান, আপনি সিফ্ট কেরাস দিয়ে এই কৌশলগুলি ব্যবহার শুরু করতে পারেন। যদি আপনি এর আগে কখনও কেরাস ব্যবহার না করে থাকেন তবে আপনি এর মাধ্যমে আপনি আরও কিছুটা জানতে পারেন [Keras documentation](https://keras.io). 

ব্যক্তিগত ভবিষ্যদ্বাণীগুলি পরিবেশন করার আগে, প্রথম পদক্ষেপটি আপনার মডেলটিকে সাধারণ কেরাস দিয়ে প্রশিক্ষণ দেওয়া। উদাহরণ হিসাবে, আমরা হস্তাক্ষর অঙ্কগুলি শ্রেণীবদ্ধ করার জন্য একটি মডেলকে প্রশিক্ষণ দেব। এই মডেলটি প্রশিক্ষণের জন্য আমরা ক্যানোনিকাল ব্যবহার করব [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

আমরা ধার নিলাম [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) রেফারেন্স কেরাস সংগ্রহশালা থেকে। আপনার শ্রেণিবদ্ধকরণের মডেলটি প্রশিক্ষণের জন্য, আপনি কেবল নীচের ঘরটি চালাবেন।

In [ ]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## ভবিষ্যতের ব্যক্তিগত ভবিষ্যদ্বাণী করার জন্য আপনার মডেলের ওজনগুলি সংরক্ষণ করুন (Save your model's weights for future private prediction)

চমৎকার আপনার মডেল প্রশিক্ষিত হয়েছে। আসুন মডেল ওজনগুলি দিয়ে সংরক্ষণ করুন`model.save()`. পরবর্তী নোটবুকে, আমরা ব্যক্তিগত পূর্বাভাসগুলি সরবরাহ করা শুরু করতে সিফ্ট কেরাসে এই ওজনগুলি লোড করব।

In [ ]:
model.save('short-conv-mnist.h5')